1. Используя Gym Atari, выбрал любую игру с более чем 4-мя действиями.
2. Обучил на стратегии градиента стратегии (градиентной политики).

# Подготовка

In [ ]:
!pip install "gymnasium[atari]"
!pip install autorom[accept-rom-license]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=dd1253f60a31ddeea12797efeda44520be883bad3753429c7850c80d65e81d68
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


### Импорт библиотек

In [ ]:
import gymnasium as gym
import ale_py
import gymnasium as gym
from gymnasium.wrappers import FlattenObservation
from gymnasium.wrappers import GrayscaleObservation
from gymnasium.wrappers import ClipAction
import torch

# Регистрация окружения
gym.register_envs(ale_py)

# Стратегия случайного поиска

In [ ]:
# Среда
env = gym.make("ALE/MsPacman-v5", render_mode='rgb_array')

# Видео
env = gym.wrappers.RecordVideo(
    env,
    episode_trigger=lambda num: num % 100 == 0,
    video_folder="random-video-folder",
    name_prefix="video-",
)

print(f'env.observation_space.shape: {env.observation_space.shape}')
print(f"env.observation_space.shape[0]: {env.observation_space.shape[0]}")
print(f"env.action_space.n: {env.action_space.n}")
print(f"env.action_space: {env.action_space}")

env.observation_space.shape: (210, 160, 3)
env.observation_space.shape[0]: 210
env.action_space.n: 9
env.action_space: Discrete(9)


In [ ]:
# Переводим среду в черно-серый цвет
grayscale_env = GrayscaleObservation(env)

print(f'env.observation_space.shape: {grayscale_env.observation_space.shape}')
print(f"env.observation_space.shape[0]: {grayscale_env.observation_space.shape[0]}")
print(f"env.action_space.n: {grayscale_env.action_space.n}")
print(f"env.action_space: {grayscale_env.action_space}")

env.observation_space.shape: (210, 160)
env.observation_space.shape[0]: 210
env.action_space.n: 9
env.action_space: Discrete(9)


In [ ]:
# Выравнивание среды
flatten_env = FlattenObservation(grayscale_env)

print(f'env.observation_space.shape: {flatten_env.observation_space.shape}')
print(f"env.observation_space.shape[0]: {flatten_env.observation_space.shape[0]}")
print(f"env.action_space.n: {flatten_env.action_space.n}")
print(f"env.action_space: {flatten_env.action_space}")

env.observation_space.shape: (33600,)
env.observation_space.shape[0]: 33600
env.action_space.n: 9
env.action_space: Discrete(9)


In [ ]:
# Получаем пространство
n_state = flatten_env.observation_space.shape[0]
# Действия
n_action = flatten_env.action_space.n
print(f"n_state: {n_state}")
print(f"n_action: {n_action}")

n_state: 33600
n_action: 9


### Функция запуска эпизода

In [ ]:
def run_episode(env, weight):
     state = env.reset()[0]
     total_reward = 0
     while True:
         state = torch.from_numpy(state).float()
        #  print(f"state: {state}")
         action = torch.argmax(torch.matmul(state, weight)) # состояния взвешиваются с помощью матрицы весов
        #  print(f"action: {action}")
        #  print(f"action.item(): {action.item()}")
         state, reward, done, truncated, _ = env.step(action.item())
         total_reward += reward
         if done or truncated:
             break
     return total_reward

In [ ]:
n_episode = 1000
best_total_reward = 0
best_weight = None
total_rewards = []

### Запуск эпизодов (игровых сессий)

In [ ]:
for episode in range(n_episode):
    weight = torch.rand(n_state, n_action) # случайные веса
    total_reward = run_episode(flatten_env, weight) # вознаграждение на случайных весах
    print('Эпизод {}: {}'.format(episode+1, total_reward))
    if total_reward > best_total_reward:
        best_weight = weight # запоминаем лучшие веса
        best_total_reward = total_reward # запоминаем вознаграждение на лучших весах
    total_rewards.append(total_reward)

Эпизод 1: 60.0
Эпизод 2: 70.0
Эпизод 3: 240.0
Эпизод 4: 280.0
Эпизод 5: 70.0
Эпизод 6: 70.0
Эпизод 7: 70.0
Эпизод 8: 60.0
Эпизод 9: 250.0
Эпизод 10: 210.0
Эпизод 11: 260.0
Эпизод 12: 420.0
Эпизод 13: 70.0
Эпизод 14: 210.0
Эпизод 15: 100.0
Эпизод 16: 60.0
Эпизод 17: 210.0
Эпизод 18: 670.0
Эпизод 19: 210.0
Эпизод 20: 90.0
Эпизод 21: 120.0
Эпизод 22: 90.0
Эпизод 23: 60.0
Эпизод 24: 60.0
Эпизод 25: 840.0
Эпизод 26: 60.0
Эпизод 27: 150.0
Эпизод 28: 70.0
Эпизод 29: 90.0
Эпизод 30: 140.0
Эпизод 31: 70.0
Эпизод 32: 70.0
Эпизод 33: 70.0
Эпизод 34: 220.0
Эпизод 35: 120.0
Эпизод 36: 200.0
Эпизод 37: 60.0
Эпизод 38: 240.0
Эпизод 39: 70.0
Эпизод 40: 60.0
Эпизод 41: 120.0
Эпизод 42: 70.0
Эпизод 43: 240.0
Эпизод 44: 80.0
Эпизод 45: 210.0
Эпизод 46: 70.0
Эпизод 47: 70.0
Эпизод 48: 60.0
Эпизод 49: 60.0
Эпизод 50: 60.0
Эпизод 51: 240.0
Эпизод 52: 210.0
Эпизод 53: 60.0
Эпизод 54: 70.0
Эпизод 55: 60.0
Эпизод 56: 60.0
Эпизод 57: 190.0
Эпизод 58: 90.0
Эпизод 59: 60.0
Эпизод 60: 70.0
Эпизод 61: 70.0
Эпизод 6

### Среднее полное вознаграждение

In [ ]:
print('Среднее полное вознаграждение в {} эпизодах: {}'.format(n_episode, sum(total_rewards) / n_episode))

Среднее полное вознаграждение в 1000 эпизодах: 162.8


## Лучшая стратегия

### Смотрим на 100 эпохах лучшую стратегию

In [ ]:
n_episode_eval = 100
total_rewards_eval = []
for episode in range(n_episode_eval):
    total_reward = run_episode(flatten_env, best_weight)
    print('Эпизод {}: {}'.format(episode+1, total_reward))
    total_rewards_eval.append(total_reward)

Эпизод 1: 1880.0
Эпизод 2: 340.0
Эпизод 3: 350.0
Эпизод 4: 500.0
Эпизод 5: 610.0
Эпизод 6: 1880.0
Эпизод 7: 420.0
Эпизод 8: 690.0
Эпизод 9: 440.0
Эпизод 10: 1880.0
Эпизод 11: 1880.0
Эпизод 12: 440.0
Эпизод 13: 350.0
Эпизод 14: 1880.0
Эпизод 15: 590.0
Эпизод 16: 500.0
Эпизод 17: 330.0
Эпизод 18: 980.0
Эпизод 19: 1880.0
Эпизод 20: 500.0
Эпизод 21: 1880.0
Эпизод 22: 980.0
Эпизод 23: 350.0
Эпизод 24: 1880.0
Эпизод 25: 1880.0
Эпизод 26: 980.0
Эпизод 27: 500.0
Эпизод 28: 450.0
Эпизод 29: 250.0
Эпизод 30: 500.0
Эпизод 31: 680.0
Эпизод 32: 1880.0
Эпизод 33: 670.0
Эпизод 34: 500.0
Эпизод 35: 1880.0
Эпизод 36: 500.0
Эпизод 37: 250.0
Эпизод 38: 1880.0
Эпизод 39: 500.0
Эпизод 40: 690.0
Эпизод 41: 1880.0
Эпизод 42: 440.0
Эпизод 43: 1880.0
Эпизод 44: 980.0
Эпизод 45: 1880.0
Эпизод 46: 590.0
Эпизод 47: 500.0
Эпизод 48: 1880.0
Эпизод 49: 700.0
Эпизод 50: 500.0
Эпизод 51: 1880.0
Эпизод 52: 1880.0
Эпизод 53: 440.0
Эпизод 54: 690.0
Эпизод 55: 700.0
Эпизод 56: 470.0
Эпизод 57: 500.0
Эпизод 58: 670.0
Эпизо

### Среднее полное вознаграждение

In [ ]:
print('Среднее полное вознаграждение в {} эпизодах: {}'.format(
          n_episode_eval, sum(total_rewards_eval) / n_episode_eval))

### График вознаграждения на этапе обучения

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_rewards)
plt.xlabel('Эпизод')
plt.ylabel('Вознаграждение')
plt.show()

# Алгоритм градиентной стратегии

In [ ]:
# Экземпляр окружающей среды
env = gym.make("ALE/MsPacman-v5", render_mode='rgb_array')

# Запись видео среды
env = gym.wrappers.RecordVideo(
    env,
    episode_trigger=lambda num: num % 100 == 0,
    video_folder="gradient-video-folder",
    name_prefix="video-",
)

# Переводим среду в черно-серый цвет
grayscale_env = GrayscaleObservation(env)

# Выравнивание среды
flatten_env = FlattenObservation(grayscale_env)

# Получаем пространство
n_state = flatten_env.observation_space.shape[0]
# Действия
n_action = flatten_env.action_space.n

### Функция запуска эпизода

In [ ]:
def run_episode(env, weight):
    state = env.reset()[0]
    grads = []
    total_reward = 0

    while True:
        # Преобразование состояния в тензор PyTorch
        state = torch.from_numpy(state).float()
        # Вычисляем логиты с использованием текущего состояния и весов.
        z = torch.matmul(state, weight)
        # Применяем функцию Softmax к логитам, чтобы получить вероятности действий
        probs = torch.nn.Softmax()(z)

        # Выбор действия на основе вероятности
        # (путем сэмплирования из бернуллиевского распределения с вероятностью probs[1])
        # Это означает, что действие будет выбрано с вероятностью probs[1] и будет равно 1, иначе 0.
        action = int(torch.argmax(probs).item())

        # Вместо бернулли мы могли бы использовать и такой выбор действий
        # Выбор действия: 0 с вероятностью probs[0], 1 с вероятностью probs[1]
        # action = 0 if probs[0] > probs[1] else 1

        # Вычисляем производную Softmax (матрицы Якоби)
        d_softmax = torch.diag(probs) - probs.view(-1, 1) * probs

        # Вычисляем производную логарифма вероятности выбранного действия
        d_log = d_softmax[action] / probs[action]

        # Вычисляем градиент весов относительно выбранного действия
        grad = state.view(-1, 1) * d_log

        # Добавляем градиент в список градиентов
        grads.append(grad)

        # Выполняем выбранное действие в среде, получаем новое состояние,
        # вознаграждение и флаг завершения эпизода
        state, reward, done, truncated, _ = env.step(action)

        # Добавляем полученное вознаграждение к общему вознаграждению
        total_reward += reward
        if done or truncated:
            break
    return total_reward, grads

In [ ]:
n_episode = 1000
weight = torch.rand(n_state, n_action)

In [ ]:
total_rewards = []
learning_rate = 0.07

### Обучение градиентной стратегии

In [ ]:
for episode in range(n_episode):
    total_reward, gradients = run_episode(flatten_env, weight)
    print('Эпизод {}: {}'.format(episode + 1, total_reward))
    for i, gradient in enumerate(gradients):
        weight += learning_rate * gradient * (total_reward - i)
    total_rewards.append(total_reward)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Эпизод 1: 810.0
Эпизод 2: 210.0
Эпизод 3: 210.0
Эпизод 4: 210.0
Эпизод 5: 210.0
Эпизод 6: 210.0
Эпизод 7: 210.0
Эпизод 8: 210.0
Эпизод 9: 210.0
Эпизод 10: 210.0
Эпизод 11: 210.0
Эпизод 12: 210.0
Эпизод 13: 210.0
Эпизод 14: 210.0
Эпизод 15: 210.0
Эпизод 16: 210.0
Эпизод 17: 210.0
Эпизод 18: 210.0
Эпизод 19: 210.0
Эпизод 20: 210.0
Эпизод 21: 210.0
Эпизод 22: 210.0
Эпизод 23: 210.0
Эпизод 24: 210.0
Эпизод 25: 210.0
Эпизод 26: 210.0
Эпизод 27: 210.0
Эпизод 28: 210.0
Эпизод 29: 210.0
Эпизод 30: 210.0
Эпизод 31: 210.0
Эпизод 32: 210.0
Эпизод 33: 210.0
Эпизод 34: 210.0
Эпизод 35: 210.0
Эпизод 36: 210.0
Эпизод 37: 210.0
Эпизод 38: 210.0
Эпизод 39: 210.0
Эпизод 40: 210.0
Эпизод 41: 210.0
Эпизод 42: 210.0
Эпизод 43: 210.0
Эпизод 44: 210.0
Эпизод 45: 210.0
Эпизод 46: 210.0
Эпизод 47: 210.0
Эпизод 48: 210.0
Эпизод 49: 210.0
Эпизод 50: 210.0
Эпизод 51: 210.0
Эпизод 52: 210.0
Эпизод 53: 210.0
Эпизод 54: 210.0
Эпизод 55: 210.0
Эпизод 56: 210.0
Эпизод 57: 210.0
Эпизод 58: 210.0
Эпизод 59: 210.0
Эпизод

# Видео

In [ ]:
import os

random_video_path = './random-video-folder'
gradient_video_path = './gradient-video-folder'

random_videos = sorted(os.listdir(random_video_path))
gradient_videos = sorted(os.listdir(gradient_video_path))

In [ ]:
best_size, best_random_path = None, None

for name in random_videos:
  temp_path = os.path.join(random_video_path, name)
  temp_size = os.path.getsize(temp_path)

  if best_size is None:
    best_size = temp_size
    best_random_path = temp_path
  elif best_size < temp_size:
    best_size = temp_size
    best_random_path = temp_path

### Видео случайной стратегии

In [ ]:
from IPython.display import HTML
from base64 import b64encode
# def view_video(path):
mp4 = open(best_random_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

### Видео градиентной стратегии

In [ ]:
mp4 = open(os.path.join(gradient_video_path, gradient_videos[-1]),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

В алгоритме градиента стратегии веса модели изменяются в направлении градиента в конце каждого эпизода. На каждом шаге алгоритм производит выборку из стратегии на основе вероятностей, вычисленных с использованием состояний и весов. Выбираемое действие определено не однозначно. Из-за этого стратегия перестает быть детерминированной, а становится стохастической.
А в стохастической среде результат не может быть определен по текущему состоянию из-за повышенной неопределенности.

Стратегия не может найти глобальную оптимальную точку. Сложно подобрать скорость обучения, так как при большой скорости - стратегия может перепрыгнуть минимумы, а при маленькой скорости обучения затраивается большое количество времени